# FT8 Mapper

### TL;DR:
Click on __Restart the kernel then re-run the whole notebook__ on the Jupiter Lab toolobar, then scroll down to the bottom of this page and click on the __Build Map__ button.

### Overview

This script was inspired by the [presentation of Jose CT1BOH](https://www.contestuniversity.com/wp-content/uploads/2021/05/There-is-Nothing-Magic-About-Propagation-CTU-2021-CT1BOH.pdf) on the use of FT8 spot data for HF propagation nowcasting. It downloads real-time data from different sources and plots them all on the same map. The data include:
- FT8 spots from the [PSKReporter web site](https://www.pskreporter.info/pskmap.html),
- MUF(3000) map from  [KC2G](https://prop.kc2g.com/) or [IZMIRAN](https://www.izmiran.ru/ionosphere/weather/daily/index.shtml),
- auroral oval from [NOAA](https://www.swpc.noaa.gov/products/aurora-30-minute-forecast),
- magnetic dip (inclination) from [NOAA](https://www.ngdc.noaa.gov/geomag/calculators/magcalc.shtml#igrfgrid),
- gray line (computed).

The map is available in the Geographic, Polar and Azimuthal projections.

The script is provided as a Jupyter notebook that includes Python code, narrative text and visual output, all in one page. Several sections below contain the code, the last section presents a number of controls that you can use to set the desired map parameters and build the map.

## License

Copyright © 2021 Alex Shovkoplyas VE3NEA

License: [MIT](https://opensource.org/licenses/MIT) (_you can do whatever you want as long as you include the original copyright and license notice_).

### Import required packages

In [1]:
import zipfile
from datetime import datetime
import re
from enum import Enum
import pylab
import cartopy.crs as ccrs
from cartopy.feature.nightshade import Nightshade
import matplotlib.pyplot as plt
import json
from IPython.display import clear_output, FileLink, FileLinks
from colorama import Fore, Style
from io import StringIO

import urllib.request as urllib
import requests
import base64
import png
import numpy as np
import matplotlib
import ipywidgets as widgets

import cartopy

### Convert grid square to lon/lat

In [2]:
letters='ABCDEFGHIJKLMNOPQRSTUVWX'
digits = '0123456789'
letters12 = dict([(letters[i] + letters[j], np.array([i*20.-180, j*10.-90])) for i in range(18) for j in range(18)])
digits34  = dict([(digits[i] + digits[j], np.array([2.*i, 1.*j])) for i in range(10) for j in range(10)])
letters56 = dict([(letters[i] + letters[j], np.array([2./24*i, 1./24*j])) for i in range(24) for j in range(24)])

def square_to_lon_lat(square):
    try:
        if len(square) == 2: return letters12[square[:2]] + [10, 5] 
        if len(square) == 4: return letters12[square[:2]] + digits34[square[2:4]] + [1, 0.5] 
        if len(square) == 6: return letters12[square[:2]] + digits34[square[2:4]] + letters56[square[4:6]] + [1./24, 0.5/24]
    except:
        return None 

### Download IZMIRAN MUF map
[Source](https://www.izmiran.ru/ionosphere/weather/daily/index.shtml)


In [3]:
headers = 'START OF foF2 MAP|START OF hmF2 MAP|EPOCH OF CURRENT MAP|LAT/LON1/LON2/DLON'

def __get_ionex(url):
    data, _ = urllib.urlretrieve(url)
    data = zipfile.ZipFile(data, 'r')
    data = data.open(data.namelist()[0])                   

    data = [line.decode('UTF-8') for line in data]
    data = [line for line in data if re.search(headers, line) is None][7:]
    data = ''.join(data)
    data = np.fromstring(data, dtype='float', sep=' ')   
    data.shape = (24, 71, 73)
    return data[datetime.utcnow().hour][-1::-1,:]


def download_muf_izmiran():
    date_string = datetime.utcnow().strftime('%y/%m/%y%m%d')
    foF2_url = f'https://www.izmiran.ru/ionosphere/weather/gram/dfc/{date_string}f1.zip'
    hmF2_url = f'https://www.izmiran.ru/ionosphere/weather/gram/dhc/{date_string}h1.zip'
    
    foF2 = __get_ionex(foF2_url) / 10
    hmF2 = __get_ionex(hmF2_url)
    
    return foF2 * 1490 / (hmF2 + 176)

### Download KC2G MUF map
[Source](https://prop.kc2g.com/)

In [4]:
MIN_MHZ = 4
MAX_MHZ = 35
DECIMATION_FACTOR = 4

# palette
cmap = pylab.cm.get_cmap('viridis', 256)  
colors = [cmap(i, alpha=0.35, bytes=True) for i in range(256)]
colors = [c[0] + (c[1] << 8) + (c[2] << 16) + (c[3] << 24) for c in colors]
color_to_mhz = dict([(colors[i], MIN_MHZ * (MAX_MHZ / MIN_MHZ)**(i/255)) for i in range(256)])

def download_muf_kc2g():
    # download
    url = 'https://prop.kc2g.com/renders/current/mufd-normal-now.svg'
    svg = requests.get(url).text

    # extract pixels
    png_data = re.search('xlink:href="data:image/png;base64, ([^"]+)"', svg).group(1)
    png_data = base64.b64decode(png_data)
    _, _, pixel_bytes, _ = png.Reader(bytes=png_data).read()
    pixel_bytes = list(pixel_bytes)[::DECIMATION_FACTOR]
    rows = [np.frombuffer(row, dtype=np.uint32) for row in pixel_bytes]

    # pixels to MHz
    return [[color_to_mhz[v] for v in row[::DECIMATION_FACTOR]] for row in rows]

### Download FT8 spots
[Source](https://www.pskreporter.info/pskmap.html)

In [5]:
bands = {    
    '5.1': '5000000-6000000', '7': '6000000-8000000', '10': '8000000-12000000', '14': '12000000-16000000', 
    '18': '16000000-19000000', '21': '19000000-22000000', '24': '22000000-26000000', '28': '27999999-31000000'
}

def download_ft8(home_square='FN03', mhz='14'):
    # download
    url = 'https://www.pskreporter.info/cgi-bin/pskquery5.pl?encap=1&callback=doNothing&statistics=1&noactive=1&nolocator=0' \
    f'&flowStartSeconds=-900&frange={bands[mhz]}&mode=FT8&modify=grid&callsign={home_square}'  
    
    xml_str = requests.get(url, timeout=180).text
    with open('./ft8.txt', 'w') as f: f.write(xml_str) # for debugging

    # parse json
    json_str = re.search('doNothing\((.+)\);', xml_str, re.DOTALL)
    json_str = json_str.group(1)
    json_struct = json.loads(json_str)
        
    # paths
    report = json_struct['receptionReport']
    paths = [[r['receiverLocator'][:6].upper(), r['senderLocator'][:6].upper()] for r in report]
    paths = np.unique([[min(p), max(p)] for p in paths], axis=0)
    paths = np.array([p for p in paths if p[0] != '' and p[1] != ''])
    
    # reporting stations
    ends = np.unique(paths.flatten())
    ends.sort()

    # non-reporting stations
    report = json_struct['activeReceiver']
    stations = [r['locator'][:6].upper() for r in report if r['mode'] == 'FT8']
    stations.sort()
    stations = np.unique(stations)
    stations = list(set(stations) - set(ends))

    # grid square to lon/lat
    paths = [[square_to_lon_lat(p[0]), square_to_lon_lat(p[1])] for p in paths]
    ends = [square_to_lon_lat(s) for s in ends]
    stations = [square_to_lon_lat(s) for s in stations]

    print(f'  paths: {len(paths)},  blue stations: {len(ends)},  gray stations: {len(stations)}')    
    return {'paths': paths, 'path_ends': ends, 'stations': stations}

### Download Aurora map
[Source](https://www.swpc.noaa.gov/products/aurora-30-minute-forecast)

In [6]:
def download_aurora():
    url = 'https://services.swpc.noaa.gov/json/ovation_aurora_latest.json'
    json_str = requests.get(url).text; 
    json_struct = json.loads(json_str)
    aurora = json_struct['coordinates']
    return np.array(aurora)[:,2].reshape(360, 181).T

### Load Dip map
[Source](https://www.ngdc.noaa.gov/geomag/calculators/magcalc.shtml#igrfgrid)

In [7]:
def load_dip():
    dip_data = np.loadtxt('igrfgridData.csv',delimiter=',')
    return np.array(dip_data)[:,4].reshape(180, 361)

### Plot the map

In [8]:
class Projection(Enum):
    Geographic = 1
    Polar = 2
    Azimuthal = 3    

def plot_map(muf_data=None, dip_data=None, aurora=None, grayline=False, spot_data=None, mhz=None, projection=Projection.Azimuthal, 
             home_square=None, time=datetime.utcnow(), all_bands_muf=True):
    # projection
    geodetic_proj = ccrs.Geodetic()
    geographic_proj = ccrs.PlateCarree()
    geographic_proj._threshold = 0.1
    home = square_to_lon_lat(home_square) if home_square != None else [0, 90]

    if projection == Projection.Geographic:
        current_proj = geographic_proj 
    elif projection == Projection.Polar:
        current_proj = ccrs.AzimuthalEquidistant(home[0], 90)
    else:
        current_proj = ccrs.AzimuthalEquidistant(home[0], home[1])
        
    # base map
    fig = plt.figure(figsize=(22, 25))
    ax = plt.axes(projection=current_proj)        
    ax.set_global()   
    ax.coastlines(resolution='110m', alpha=0.9)
    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linestyle='--', alpha=0.8)
    if grayline: ax.add_feature(Nightshade(time), alpha=0.2, color='black', edgecolor='red')

    # title
    title = time.strftime('%Y-%m-%d %H:%M UTC')
    if mhz is not None: title = f'{title}  {mhz} MHz'
    if home_square is not None: title = f'{title}  {home_square}'
    plt.title(title + '\n', fontsize=15, pad=0.02)

    # MUF
    if not muf_data is None:
        lons, lats = np.meshgrid(np.linspace(-180, 180, len(muf_data[0])), np.linspace(-90,90, len(muf_data)))
        levels = [4,5.3,7,10.1,14,18,21,24.8,28,35] if all_bands_muf else [4, mhz, 35]
        # filled countours
        contours = plt.contourf(lons, lats, muf_data, levels=levels, transform=geographic_proj, cmap='jet', alpha=0.5)
        plt.colorbar(orientation='horizontal', shrink=0.3, pad=0.02).ax.set_xlabel('MUF(3000), MHz')        
        # labels
        isolines = plt.contour(lons, lats, muf_data, levels=levels, colors=['None'], transform=geographic_proj)
        ax.clabel(isolines, colors=['gray'], manual=False, inline=True, fmt=' {:.0f} '.format)

    # dip
    if not dip_data is None:
        lons, lats = np.meshgrid(np.linspace(-180, 180, 361), np.linspace(89,-90, 180))
        levels = [-90,-80,-70,-60,-50,-40,-30,-20,-10,0,10,20,30,40,50,60,70,80,90]
        # contours
        isolines = plt.contour(lons, lats, dip_data, levels=levels, alpha=.5, linewidths=[.5, .5, .5, .5] , colors='#020599', transform=geographic_proj)
        # labels
        isolines = plt.contour(lons, lats, dip_data, levels=levels, colors=['None'], transform=geographic_proj)
        ax.clabel(isolines, colors='grey', manual=False, inline=True, fmt=' {:.0f} '.format)
        
    # aurora
    if aurora is not None:
        ax.imshow(aurora, transform=geographic_proj, extent=(0, 360, 90, -90), cmap='Greys', vmin=1, vmax=10)
        
    # spots
    if not spot_data is None:
        ax.scatter([s[0] for s in spot_data['stations']], [s[1] for s in spot_data['stations']], s=60, color='silver', edgecolors='gray', transform=geographic_proj, zorder=3)        
        for p in spot_data['paths']: plt.plot([p[0][0], p[1][0]], [p[0][1], p[1][1]], color='blue', transform=geodetic_proj, zorder=4)
        ax.scatter([s[0] for s in spot_data['path_ends']], [s[1] for s in spot_data['path_ends']], s=60, color='aqua', edgecolors='blue', transform=geographic_proj, zorder=5)

    # grid
    if home_square is None: 
        # no home location, plot lat/lon
        xlocs=np.linspace(-180,180,19)
        ylocs=np.linspace(-90,90,19)
        grid = ax.gridlines(color='magenta', alpha=0.7, xlocs=xlocs, ylocs=ylocs)
    else:
        if projection == Projection.Geographic:
            ax2 = fig.add_axes(ax.get_position(), projection=ccrs.RotatedPole(central_rotated_longitude=home[0] + 180, pole_latitude=-home[1]))                    
        elif projection == Projection.Polar:
            ax2 = fig.add_axes(ax.get_position(), projection=ccrs.AzimuthalEquidistant(0, -home[1]))        
        else:
            ax2 = fig.add_axes(ax.get_position(), projection=ccrs.AzimuthalEquidistant(0, -90))        
        
    ax2.set_global()
    ax2.patch.set_facecolor('none')
    xlocs=np.linspace(-180,180,13)
    ylocs=np.linspace(77,-90,7)
    grid = ax2.gridlines(color='teal', alpha=0.7, xlocs=xlocs, ylocs=ylocs)
    ax.plot(home[0], home[1], marker='*', color='yellow', mec='olive', ms=20, zorder=9, transform=geographic_proj)
        
    grid.n_steps = 1000
    
    plt.savefig('map.png')
    plt.show()

### User interface

In [9]:
ui = {}
muf_data = None
spot_data = None
dip_data = None
aurora = None

In [10]:
def show_widgets():
    # widgets
    options = [('Geographic', Projection.Geographic), ('Polar', Projection.Polar), ('Azimuthal', Projection.Azimuthal)]
    ui['projection'] = widgets.RadioButtons(options=options, value=Projection.Polar, description='Projection:')
    ui['mhz'] = widgets.Dropdown(options=['5.3', '7', '10.1', '14', '18', '21', '24.8', '28'], value='14', description='MHz:')
    
    ui['home_square'] = widgets.Text(value='FN03', description='Square:', placeholder='All squares')
    ui['square_valid'] = widgets.Valid(value=True)
    ui['square'] = widgets.Box([ui['home_square'], ui['square_valid']])
    
    ui['show_spots'] = widgets.Checkbox(value=True, description='Show FT8 Spots', indent=False)
    ui['show_muf'] = widgets.Checkbox(value=True, description='Show MUF(3000) Map', disabled=False, indent=False)
    ui['all_bands_muf'] = widgets.Checkbox(value=True, description='MUF for All Bands', disabled=False, indent=False)
    ui['show_aurora'] = widgets.Checkbox(description='Show Aurora', indent=False)
    ui['show_dip'] = widgets.Checkbox(description='Show Dip', indent=False)
    ui['show_grayline'] = widgets.Checkbox(description='Show Grayline', indent=False)

    ui['download_spots'] = widgets.Checkbox(description='Re-download Spot Data', indent=False)
    ui['download_aurora'] = widgets.Checkbox(description='Re-download Auroral Data', indent=False)
    ui['download_muf'] = widgets.Checkbox(description='Re-download MUF Data', indent=False)
    ui['muf_source'] = widgets.RadioButtons(options=['KC2G', 'IZMIRAN'], description='MUF Source:')
    ui['button'] = widgets.Button(description='Build Map')

    # layout
    ui['box1'] = widgets.VBox([ui['projection'], ui['mhz'], ui['square']])
    ui['box2']= widgets.VBox([ui['show_spots'], ui['show_muf'], ui['all_bands_muf'], ui['show_dip'], ui['show_aurora'], ui['show_grayline']])
    ui['box3'] = widgets.VBox([ui['download_spots'], ui['download_aurora'], ui['download_muf'], ui['muf_source']])

    layout = widgets.Layout(height='200px', width='100%', border='1px solid gray')
    display(widgets.HBox(children=[ui['box1'], ui['box2'], ui['box3']], layout=layout))
    layout = widgets.Layout(height='70px', width='100%', align_items='center')
    display(widgets.HBox(children=[ui['button']], layout=layout))
                                   
    # output
    ui['out'] = widgets.Output()
    display(ui['out'])

    # callbacks
    ui['home_square'].observe(on_text_change, names='value')
    ui['button'].on_click(on_button_click)
    
    
def download_data():
    global muf_data, spot_data, dip_data, aurora
    
    if ui['download_aurora'].value or (ui['show_aurora'].value and aurora is None): 
        print('Downloading aurora...')
        try:
            aurora = download_aurora()    
        except: 
            print(Fore.RED + '  download failed' + Style.RESET_ALL)
        
    if ui['download_muf'].value or (ui['show_muf'].value and muf_data is None): 
        print('Downloading muf...')
        try:
            muf_data = download_muf_kc2g() if ui['muf_source'].value == 'KC2G' else download_muf_izmiran() 
        except: 
            print(Fore.RED + '  download failed' + Style.RESET_ALL)

    if ui['show_dip'].value and dip_data is None: 
        print('Loading dip...')
        try:
            dip_data = load_dip()
        except: 
            print(Fore.RED + '  dip load failed' + Style.RESET_ALL)
        
    if ui['download_spots'].value or (ui['show_spots'].value and spot_data is None): 
        print('Downloading spots...')
        try: 
            spot_data = download_ft8(home_square=ui['home_square'].value.upper(), mhz=ui['mhz'].value) 
        except: 
            print(Fore.RED + '  download failed' + Style.RESET_ALL)
        
    ui['download_spots'].value = False
    ui['download_muf'].value = False
    ui['download_aurora'].value = False        
    
    
def on_text_change(change):
    square = change['new'].upper()
    if ui['square_valid'] is not None:
        ui['square_valid'].value = square == '' or square_to_lon_lat(square) is not None

        
def on_button_click(button):
    with ui['out']:
        clear_output(True)
        download_data()
        
        print('Building the map...')

        plot_map(
            muf_data = muf_data if ui['show_muf'].value else None, 
            dip_data = dip_data if ui['show_dip'].value else None, 
            aurora = aurora if ui['show_aurora'].value else None, 
            spot_data = spot_data if ui['show_spots'].value else None, 
            grayline = ui['show_grayline'].value, 
            mhz=ui['mhz'].value,  
            projection = ui['projection'].value,
            home_square=ui['home_square'].value,
            all_bands_muf=ui['all_bands_muf'].value
        )

### Enter the settings and build the map
__Note:__ By default, the script downloads every dataset only once. If you change some parameter that affects the data, e.g, select a different band or different MUF source, tick the correspoiding _Re-download_ checkbox before building the map. To keep the map current, re-download all data every 15 minutes.

The map will be shown below and saved to the [map.png](./map.png) file.

In [11]:
show_widgets()

Output()